In [ ]:
import numpy as np
import os
import pandas as pd
from pathlib import Path
import requests

from functions.processing.recordings import processSingleRecording, processSingleRecordingPoint
import ffmpeg

from concurrent.futures import ThreadPoolExecutor
import noisereduce as nr
import torch
from tqdm.notebook import tqdm
from functools import partial
from functions.processing.recordings import get_noise_profile, process_and_save_as_pt

# Set the base directory to your target folder
base_directory = r'F:\\Persönliches\\Git\BioOTon\\XenoCanto'
os.makedirs(base_directory, exist_ok=True)

In [41]:
# Get number of pages with the results
api_url = f"https://xeno-canto.org/api/3/recordings?query=cnt:germany+len:30-60+grp:birds&key=7894b51f8bcdf05cfac66a1455cbb314c4313486"
print(f"→ Querying {api_url}")
response = requests.get(api_url)
if response.status_code != 200:
    print(f"API request failed.")

# process response to get page numbers
data = response.json()
page_number = data["numPages"]

recordings_list = []

# Build frame with all recording ids and lat / lon
for p in np.arange(1, page_number+1):
    api_url = f"https://xeno-canto.org/api/3/recordings?query=cnt:germany+len:30-60+grp:birds&key=7894b51f8bcdf05cfac66a1455cbb314c4313486&page={p}"
    print(f"→ Querying {api_url}")
    response = requests.get(api_url)
    if response.status_code != 200:
        print(f"API request failed.")

    data = response.json()
    recordings = data.get('recordings', [])

    
    if not recordings:
        print(f"No recordings found.")

    recordings = pd.DataFrame.from_dict(recordings)

    recordings_list.append(recordings)

recordings_frame = pd.concat(recordings_list, ignore_index=True)
recordings_frame = recordings_frame.rename(columns={"lon":"lng"})

→ Querying https://xeno-canto.org/api/3/recordings?query=cnt:germany+len:30-60+grp:birds&key=7894b51f8bcdf05cfac66a1455cbb314c4313486
→ Querying https://xeno-canto.org/api/3/recordings?query=cnt:germany+len:30-60+grp:birds&key=7894b51f8bcdf05cfac66a1455cbb314c4313486&page=1
→ Querying https://xeno-canto.org/api/3/recordings?query=cnt:germany+len:30-60+grp:birds&key=7894b51f8bcdf05cfac66a1455cbb314c4313486&page=2
→ Querying https://xeno-canto.org/api/3/recordings?query=cnt:germany+len:30-60+grp:birds&key=7894b51f8bcdf05cfac66a1455cbb314c4313486&page=3
→ Querying https://xeno-canto.org/api/3/recordings?query=cnt:germany+len:30-60+grp:birds&key=7894b51f8bcdf05cfac66a1455cbb314c4313486&page=4
→ Querying https://xeno-canto.org/api/3/recordings?query=cnt:germany+len:30-60+grp:birds&key=7894b51f8bcdf05cfac66a1455cbb314c4313486&page=5
→ Querying https://xeno-canto.org/api/3/recordings?query=cnt:germany+len:30-60+grp:birds&key=7894b51f8bcdf05cfac66a1455cbb314c4313486&page=6
→ Querying https://x

In [ ]:
# pd.DataFrame.to_parquet(recordings_frame, Path(base_directory, "xeno_canto.parquet"))

In [117]:
total_downloaded = 0
for recording_id in recordings_frame.id:
    try:
        # Download URL using the recording ID
        file_url = f"https://xeno-canto.org/{recording_id}/download"
        filename = f"{recording_id}_audio.flac"
        save_path = os.path.join(base_directory, filename)
        
        with requests.get(file_url, stream=True) as r:
            if r.status_code == 200:
                with open(save_path, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
                print(f"Downloaded {filename}")
                total_downloaded += 1
            else:
                print(f"Failed to download {filename} (status: {r.status_code})")
    except Exception as e:
        print(f"Error downloading recording {recording_id}: {e}")

Downloaded 1048296_audio.flac
Downloaded 339847_audio.flac
Downloaded 807649_audio.flac
Downloaded 597408_audio.flac
Downloaded 383534_audio.flac
Downloaded 378738_audio.flac
Downloaded 722721_audio.flac
Downloaded 710603_audio.flac
Downloaded 75654_audio.flac
Downloaded 45927_audio.flac
Downloaded 41626_audio.flac
Downloaded 927767_audio.flac
Downloaded 829070_audio.flac
Downloaded 636569_audio.flac
Downloaded 416534_audio.flac
Downloaded 840964_audio.flac
Downloaded 831858_audio.flac
Downloaded 807065_audio.flac
Downloaded 405452_audio.flac
Downloaded 354586_audio.flac
Downloaded 839633_audio.flac
Downloaded 434981_audio.flac
Downloaded 733289_audio.flac
Downloaded 485869_audio.flac
Downloaded 463138_audio.flac
Downloaded 418892_audio.flac
Downloaded 406706_audio.flac
Downloaded 71070_audio.flac
Downloaded 854587_audio.flac
Downloaded 831329_audio.flac
Downloaded 825875_audio.flac
Downloaded 711438_audio.flac
Downloaded 637503_audio.flac
Downloaded 607409_audio.flac
Downloaded 541447

In [16]:
dir_files = os.listdir(base_directory)
dir_filess = os.listdir(Path("./AudioTensors"))

ids_pt = {filename.split('_')[0] for filename in dir_filess}

duplicates = []
for filename in dir_files:
    file_id = filename.split('_')[0]
    if file_id in ids_pt:
        duplicates.append(file_id)

print(f"Duplicate IDs found: {duplicates}")

Duplicate IDs found: []


In [45]:
points, dbf_df, geo_df, joined_dfs, raster_crs = processSingleRecordingPoint(corine_dir = "F:\\Persönliches\\Git\\BioOTon", recording_dir = base_directory, x_range='all')

Loading raster.
Loaded raster successfully. 
Clipped raster to the point. 
Generating individual pixel.
   value                                           geometry  Value  \
0     27  POLYGON ((4186600 3409900, 4186600 3409800, 41...     27   

        Count               LABEL3      Red  Green      Blue CODE_18  
0  17478178.0  Moors and heathland  0.65098    1.0  0.501961     322  
Assigned class 322 to point 0
Clipped raster to the point. 
Generating individual pixel.
   value                                           geometry  Value      Count  \
0     39  POLYGON ((4214800 3531400, 4214800 3531300, 42...     39  1218413.0   

             LABEL3      Red    Green      Blue CODE_18  
0  Intertidal flats  0.65098  0.65098  0.901961     423  
Assigned class 423 to point 1
Clipped raster to the point. 
Generating individual pixel.
   value                                           geometry  Value  \
0     27  POLYGON ((4228800 3418400, 4228800 3418300, 42...     27   

        Count  

f:\Persönliches\Git\BioOTon\functions\processing\recordings.py:303: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '322' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  points.loc[points.index==idx, 'label'] = weighted_class


Clipped raster to the point. 
Generating individual pixel.
   value                                           geometry  Value      Count  \
0     11  POLYGON ((4115800 3113600, 4115800 3113500, 41...     11  1310736.0   

                         LABEL3  Red     Green  Blue CODE_18  
0  Sport and leisure facilities  1.0  0.901961   1.0     142  
Assigned class 142 to point 6
Clipped raster to the point. 
Generating individual pixel.
   value                                           geometry  Value     Count  \
0     10  POLYGON ((4306500 3333600, 4306500 3333500, 43...     10  330596.0   

              LABEL3  Red    Green  Blue CODE_18  
0  Green urban areas  1.0  0.65098   1.0     141  
Assigned class 141 to point 7
Clipped raster to the point. 
Generating individual pixel.
   value                                           geometry  Value  \
0     18  POLYGON ((4265500 3447400, 4265500 3447300, 42...     18   

        Count    LABEL3       Red     Green      Blue CODE_18  
0  430

In [ ]:
# points.to_parquet("./xeno_points_single.parquet")

In [2]:
def denoise_flac_data(file_path, output_dir, sampling_rate=16000, window_duration=2.5, target_sr=16000, target_loudness=-16):
    try:
        target_path = output_dir / (file_path.stem + "_dn.pt")

        # Skip the file if it already exists
        if target_path.exists():
            return True
        
        # Load flac file and process it 
        out, _ = (
            ffmpeg
            .input(str(file_path))
            # Normalize the Loudness
            .filter('loudnorm', i=target_loudness, tp=-1.0, lra=11)
            # ac=1: Convert to mono, ar: sample at target sr, 'f32le': output 32-bit float Little Endian
            .output('pipe:', format='f32le', acodec='pcm_f32le', ac=1, ar=target_sr)
            .run(capture_stdout=True, capture_stderr=True, quiet=True)
        )

        # Convert bytes to numpy array
        audio_np = np.frombuffer(out, np.float32)

        # Convert to 16-bit Float tensor
        wave = torch.from_numpy(audio_np.copy()).to(torch.float16)
        
        noise_part = get_noise_profile(wave, sampling_rate, window_duration)


        wave_np = wave.numpy()
        noise_np = noise_part.numpy()

        # Reduce noise
        reduced_noise = nr.reduce_noise(
            y=wave_np, 
            sr=sampling_rate, 
            y_noise=noise_np, 
            n_fft=4096,
            hop_length=204, # Approx 95% overlap (4096 * 0.05)
            prop_decrease=1.0
        )

        # Convert back
        reduced_noise_tensor = torch.from_numpy(reduced_noise).bfloat16()
        torch.save(reduced_noise_tensor, target_path)
        
        return True
    except Exception as e:
        # Returning the error string helps debugging
        return str(e)

In [4]:
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"


input_dir = Path("./XenoCanto")
output_dir = Path("./XenoCanto_denoised")
output_dir.mkdir(exist_ok=True)

audio_files = list(input_dir.glob("*.flac"))
print(f"Total files to process: {len(audio_files)}")

max_workers = max(1, os.cpu_count() - 2)

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    func = partial(denoise_flac_data, output_dir=output_dir, 
                    sampling_rate=16000, window_duration=2.5)

    results = list(tqdm(
        executor.map(func, audio_files), 
        total=len(audio_files),
        desc="Denoising data"
    ))

# Check errors
failures = [r for r in results if r is not True]
print(f"Success: {len(audio_files) - len(failures)} | Failed: {len(failures)}")
if failures:
    print(f"First error: {failures[0]}")

Total files to process: 6895


Denoising data:   0%|          | 0/6895 [00:00<?, ?it/s]

Success: 6895 | Failed: 0


In [4]:
input_dir = Path(base_directory)
output_dir = Path(f"{base_directory}+Tensors")
output_dir.mkdir(exist_ok=True)

audio_files = list(input_dir.glob("*.flac"))

print(f"Total files to process: {len(audio_files)}")

# --- Execution ---
with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    results = list(tqdm(
        executor.map(lambda f: process_and_save_as_pt(f, output_dir, target_sr=16000), audio_files), 
        total=len(audio_files),
        desc="Converting to .pt"
    ))

print(f"Success: {sum(results)} | Failed: {len(results) - sum(results)}")

Total files to process: 6895


Converting to .pt:   0%|          | 0/6895 [00:00<?, ?it/s]

Success: 6895 | Failed: 0
